<h1> Create TensorFlow model </h1>

This notebook illustrates:
<ol>
<li> Creating a model using the high-level Estimator API 
</ol>

In [126]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-b602001d56524322'
PROJECT = 'qwiklabs-gcp-b602001d56524322'
REGION = 'us-central1'

In [127]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [128]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create TensorFlow model using TensorFlow's Estimator API </h2>
<p>
First, write an input_fn to read the data.
<p>

## Lab Task 1
Verify that the headers match your CSV output

In [129]:
import shutil
import numpy as np
import tensorflow as tf

In [130]:
# Determine CSV, label, and key columns
CSV_COLUMNS = 'weight_pounds,is_male,mother_age,plurality,gestation_weeks,key'.split(',')
LABEL_COLUMN = 'weight_pounds'
KEY_COLUMN = 'key'

# Set default values for each CSV column
DEFAULTS = [[0.0], ['null'], [0.0], ['null'], [0.0], ['nokey']]
TRAIN_STEPS = 1000

## Lab Task 2

Fill out the details of the input function below

In [131]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def read_dataset(filename_pattern, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(line_of_text):
      # TODO #1: Use tf.decode_csv to parse the provided line
      columns = tf.decode_csv(line_of_text, record_defaults=DEFAULTS)
      print ("columns: ", columns)
      # TODO #2: Make a Python dict.  The keys are the column names, the values are from the parsed data
      features = dict(zip(CSV_COLUMNS,columns))
      print ("features: ", features)
      # TODO #3: Return a tuple of features, label where features is a Python dict and label a float
      label = features.pop(LABEL_COLUMN)
      print ("label: ", label)
      return features, label
    
    # TODO #4: Use tf.gfile.Glob to create list of files that match pattern
    file_list = tf.gfile.Glob(filename_pattern)

    # Create dataset from file list
    dataset = (tf.data.TextLineDataset(file_list)  # Read text file
                 .map(decode_csv))  # Transform each elem by applying decode_csv fn
    
    # TODO #5: In training mode, shuffle the dataset and repeat indefinitely
    #                (Look at the API for tf.data.dataset shuffle)
    #          The mode input variable will be tf.estimator.ModeKeys.TRAIN if in training mode
    #          Tell the dataset to provide data in batches of batch_size 
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epocs = None # indefinitely
        dataset = dataset.shuffle(buffer_size=10*batch_size)
    else:
        num_epocs = 1 # end-of-input after this
    dataset = dataset.repeat(num_epocs).batch(batch_size)
    
    # This will now return batches of features, label
    return dataset
  return _input_fn

## Lab Task 3

Use the TensorFlow feature column API to define appropriate feature columns for your raw features that come from the CSV.

<b> Bonus: </b> Separate your columns into wide columns (categorical, discrete, etc.) and deep columns (numeric, embedding, etc.)

In [132]:
# Define feature columns
def get_categorical(name, values):
  return tf.feature_column.indicator_column(
    tf.feature_column.categorical_column_with_vocabulary_list(name, values))

def get_cols():
  # Define column types
  return [\
          get_categorical('is_male', ['True', 'False', 'Unknown']),
          tf.feature_column.numeric_column('mother_age'),
          get_categorical('plurality',
                      ['Single(1)', 'Twins(2)', 'Triplets(3)',
                       'Quadruplets(4)', 'Quintuplets(5)','Multiple(2+)']),
          tf.feature_column.numeric_column('gestation_weeks')
      ]

## Lab Task 4

To predict with the TensorFlow model, we also need a serving input function (we'll use this in a later lab). We will want all the inputs from our user.

Verify and change the column names and types here as appropriate. These should match your CSV_COLUMNS

In [133]:
# Create serving input function to be able to serve predictions later using provided inputs
def serving_input_fn():
    feature_placeholders = {
        'is_male': tf.placeholder(tf.string, [None]),
        'mother_age': tf.placeholder(tf.float32, [None]),
        'plurality': tf.placeholder(tf.string, [None]),
        'gestation_weeks': tf.placeholder(tf.float32, [None])
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

## Lab Task 5

Complete the TODOs in this code:

In [134]:
# Create estimator to train and evaluate
def train_and_evaluate(output_dir):
  EVAL_INTERVAL = 300
  run_config = tf.estimator.RunConfig(save_checkpoints_secs = EVAL_INTERVAL,
                                      keep_checkpoint_max = 3)
  # TODO #1: Create your estimator
  column =  tf.feature_column.numeric_column('weight_pounds')
  estimator = tf.estimator.DNNRegressor(
                       model_dir = output_dir,
                       feature_columns = get_cols(),
                       hidden_units = [64, 32],
                       config = run_config)
  train_spec = tf.estimator.TrainSpec(
                       # TODO #2: Call read_dataset passing in the training CSV file and the appropriate mode
                       input_fn = read_dataset('/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/06_structured/train*',mode=tf.contrib.learn.ModeKeys.EVAL),
                       max_steps = TRAIN_STEPS)
  exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  eval_spec = tf.estimator.EvalSpec(
                       # TODO #3: Call read_dataset passing in the evaluation CSV file and the appropriate mode
                       input_fn = read_dataset('/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/06_structured/valid*',mode=tf.contrib.learn.ModeKeys.EVAL),
                       steps = None,
                       start_delay_secs = 60, # start evaluating after N seconds
                       throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
                       exporters = exporter)
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Finally, train!

In [135]:
# Run the model
shutil.rmtree('babyweight_trained', ignore_errors = True) # start fresh each time
tf.summary.FileWriterCache.clear() # ensure filewriter cache is clear for TensorBoard events file
train_and_evaluate('babyweight_trained')

INFO:tensorflow:Using config: {'_master': '', '_session_config': None, '_model_dir': 'babyweight_trained', '_num_ps_replicas': 0, '_task_type': 'worker', '_is_chief': True, '_save_checkpoints_secs': 300, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_service': None, '_evaluation_master': '', '_task_id': 0, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc5c5c5bda0>, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_train_distribute': None, '_keep_checkpoint_max': 3, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 300 secs (eval_spec.throttle_secs) or training is finished.
columns:  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf

INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-106
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b"babyweight_trained/export/exporter/temp-b'1553880977'/saved_model.pb"
columns:  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>]
features:  {'mother_age': <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, 'key': <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>, 'weight_pounds': <tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, 'is_male': <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, 'plurality': <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, 'gestation_weeks': <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>}
label:  Tensor("DecodeCSV:0", shape=(), 

columns:  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>]
features:  {'mother_age': <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, 'key': <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>, 'weight_pounds': <tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, 'is_male': <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, 'plurality': <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, 'gestation_weeks': <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>}
label:  Tensor("DecodeCSV:0", shape=(), dtype=float32)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-212
INFO:tensorflow:R

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-318
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 319 into babyweight_trained/model.ckpt.
INFO:tensorflow:step = 319, loss = 639.12195
INFO:tensorflow:Saving checkpoints for 371 into babyweight_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 156.51509.
columns:  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>]
features:  {'mother_age': <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, 'key': <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>, 'weight_pounds': <tf.Tensor 'D

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 425 into babyweight_trained/model.ckpt.
INFO:tensorflow:step = 425, loss = 633.31433
INFO:tensorflow:Saving checkpoints for 477 into babyweight_trained/model.ckpt.
INFO:tensorflow:Loss for final step: 156.23477.
columns:  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>]
features:  {'mother_age': <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, 'key': <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>, 'weight_pounds': <tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, 'is_male': <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, 'plurality': <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, 'gestation_weeks': <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>}
label: 

INFO:tensorflow:Loss for final step: 156.06801.
columns:  [<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>]
features:  {'mother_age': <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, 'key': <tf.Tensor 'DecodeCSV:5' shape=() dtype=string>, 'weight_pounds': <tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, 'is_male': <tf.Tensor 'DecodeCSV:1' shape=() dtype=string>, 'plurality': <tf.Tensor 'DecodeCSV:3' shape=() dtype=string>, 'gestation_weeks': <tf.Tensor 'DecodeCSV:4' shape=() dtype=float32>}
label:  Tensor("DecodeCSV:0", shape=(), dtype=float32)
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-29-17:36:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring para

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-29-17:36:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-689
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-29-17:36:57
INFO:tensorflow:Saving dict for global step 689: average_loss = 0.0, global_step = 689, loss = 0.0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'mother_age': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'plurality

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-29-17:37:04
INFO:tensorflow:Saving dict for global step 795: average_loss = 0.0, global_step = 795, loss = 0.0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'mother_age': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'plurality': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=string>, 'gestation_weeks': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=float32>, 'is_male': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>}
INFO:tensorflow:'serving_default' : Regression input 

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'mother_age': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'plurality': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=string>, 'gestation_weeks': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=float32>, 'is_male': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>}
INFO:tensorflow:'serving_default' : Regression input must be a single string Tensor; got {'mother_age': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'plurality': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=string>, 'gestation_weeks': <tf.Tensor 'Placeholder_3:0' shape=(?,) dt

INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'mother_age': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'plurality': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=string>, 'gestation_weeks': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=float32>, 'is_male': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>}
INFO:tensorflow:'serving_default' : Regression input must be a single string Tensor; got {'mother_age': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'plurality': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=string>, 'gestation_weeks': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=float32>, 'is_male': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=string>}
INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-1000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tens

When I ran it, the final lines of the output (above) were:
<pre>
INFO:tensorflow:Saving dict for global step 1000: average_loss = 1.2693067, global_step = 1000, loss = 635.9226
INFO:tensorflow:Restoring parameters from babyweight_trained/model.ckpt-1000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: babyweight_trained/export/exporter/temp-1517899936/saved_model.pb
</pre>
The exporter directory contains the final model and the final RMSE (the average_loss) is 1.2693067

<h2> Monitor and experiment with training </h2>

In [138]:
from google.datalab.ml import TensorBoard
TensorBoard().start('./babyweight_trained')

TensorBoard was started successfully with pid 17415. Click here to access it.

17415

In [137]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 17384


Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License